In [1]:
# Important! This notebook runs on Python 3

import pandas as pd
import numpy as np
from numpy import random
import time

# Visualization
from bokeh.plotting import figure, output_file, show
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
# Necessary world information

# Prices and Costs
# Prices of one beer at each level of the supply chain.
retail_price = 200
wholesale_price = 9
regional_warehouse_price = 8
factory_price = 7
field_price = 6
# Cost of holding one beer during one day on warehouse.
# Assumed to be the same for all levels
warehouse_price = 0.5
# Cost of backlog: non fulfilled orders
backlog_cost = 0.01

# Initial Inventories
retail_ininv = 100
wholesale_ininv = 100
regional_warehouse_ininv = 100
factory_ininv = 100

In [3]:
class Customer:
    """
    This type of agent doesn't learn, just interacts with Retail by demanding beer
    """
    def __init__(self,demand_trend):
        self.demand = demand_trend['Demand']

class Fields:
    """
    This type of agent doesn't learn, just interacts with Factory by supplying beer
    """
    def __init__(self,supply_trend):
        self.supply = supply_trend['Supply']  

class Agent:
    """
    Creates a Beer Supply Chain Agent ready to start interacting
    with other agents and learn.
    input:
    * name (string) indicating the type of agent, can be one of four:
    {Retail,Wholesale,Regional_Warehouse,Factory} 
    * inventory (numeric) starting inventory at day 1 
    output: an object of type Agent
    """
    def __init__(self,name,inventory):
        
        # I am letting different levels have different selling and buying prices
        # This could also include different warehousing/backlogs costs
        if name == "Retail":
            self.selling_price = retail_price
            self.buying_price = wholesale_price
        elif name == "Wholesale":
            self.selling_price = wholesale_price
            self.buying_price = regional_warehouse_price
        elif name == "Regional_Warehouse":
            self.selling_price = regional_warehouse_price
            self.buying_price = factory_price
        elif name == "Factory":
            self.selling_price = factory_price
            self.buying_price = field_price
        
        self.name = name
        self.initial_inventory = inventory
        self.inventory = inventory
        self.total_warehousing_costs = 0
        self.total_money = 0
        self.backlog = 0
        self.current_policy = [0] * 365
        self.current_payout = [0] * 365
        self.best_policy = [0] * 365
        self.best_payout = [-100000000] * 365
        self.historic_payout = []
        
        # These relationships are assigned after the agents are created
        self.downstream_agent =  ""
        self.upstream_agent =  ""
    
    def pay_for_warehousing(self):
        # Pays for warehousing of inventory: must be done either
        # "first thing in the morning" or "last time in the night"
        self.total_money = self.total_money - \
                (self.inventory * warehouse_price)
    
    def receive_upstream(self,orders):
        # Receives orders from upstream agent first thing in the morning
        self.inventory = self.inventory + orders
        self.total_money = self.total_money - \
                (orders * self.buying_price)
        
    def give_downstream(self,orders):
        # Checks if he has availability to fulfill order,
        # fulfills as much as he can
        if self.inventory >= orders:
            self.total_money = self.total_money + \
                (orders * self.selling_price)
            self.inventory = self.inventory - orders
            return orders
        else:
            orders_that_could_be_fulfilled = self.inventory
            # Sells all its inventory
            self.total_money = self.total_money + \
                (orders_that_could_be_fulfilled * self.selling_price)
            # If there were non fulfilled orders, those cause a penalty
            self.backlog = (orders - self.inventory) * backlog_cost
            self.total_money = self.total_money - self.backlog
            self.inventory = 0
            return orders_that_could_be_fulfilled

# Creating the world 

We need a customer with a yearly demand trend, and fields with a yearly supply trend.

After that, we need to create the agents that will comprise our supply chain, and then assign the relationships (upstream, downstream) between them.

In [4]:
# Creating the world! Setting supply and demand trends, assigning interactions between agents

# Getting customer_demand and field_supply trends
# TODO: Both should be added a small random effect each time (iteration/epoch)
customer_demand = pd.read_csv("../aux_documents/customer_trend.csv")
fields_supply = pd.read_csv("../aux_documents/fields_trend.csv")

# Creating Supply Chain Agents
customer_agent = Customer(customer_demand)
retail_agent = Agent("Retail", retail_ininv)
wholesale_agent = Agent("Wholesale", wholesale_ininv)
regional_warehouse_agent = Agent("Regional_Warehouse", regional_warehouse_ininv)
factory_agent = Agent("Factory",factory_ininv)
fields_agent = Fields(fields_supply)

# Assigning interactions
retail_agent.downstream_agent = customer_agent
retail_agent.upstream_agent = wholesale_agent
wholesale_agent.downstream_agent = retail_agent
wholesale_agent.upstream_agent = regional_warehouse_agent
regional_warehouse_agent.downstream_agent = wholesale_agent
regional_warehouse_agent.upstream_agent = factory_agent
factory_agent.downstream_agent = regional_warehouse_agent
factory_agent.upstream_agent = fields_agent

Let's take a look at the supply and demand trends!

In [5]:
p_cd = figure(title="Customer Demand - weekly trend with a peak on Independence Day and increased demand on Christmas Holidays", plot_height=450, plot_width=900, x_range = (0,365), y_range=(0,6))
r_cd = p_cd.line(range(365), customer_agent.demand, color="seagreen")
p_cd.xaxis.axis_label = "Day"
p_cd.yaxis.axis_label = "Demand"

In [6]:
show(p_cd, notebook_handle=True)

In [7]:
p_fs = figure(title="Fields Supply - it's only produced in summertime, with a small lift near the end of June and a big lift during August", plot_height=450, plot_width=900, x_range = (0,365), y_range=(0,73))
r_fs = p_fs.line(range(365), fields_agent.supply, color="seagreen")
p_fs.xaxis.axis_label = "Day"
p_fs.yaxis.axis_label = "Production"

In [8]:
show(p_fs, notebook_handle=True)

# Policy Iteration

Basic idea of policy iteration:

    1. Start all agents with the [0]*365 policy, this would be just selling what they have and never restocking or making any decisions. This is the starting benchmark (best policy).
    2. For each agent, for every day of the year, repeat `total_epochs` times:
        2.1 Create a random (epsilon-greedy based) policy (upstream demand) for the day
        3.1 Next morning: make all transactions based on that demand
        4.1 Evaluate the payout of that policy. If the payout is higher than the payout of the best policy, it becomes the new best policy; else, nothing changes.
    
The code will print out 20 cuts in time to show a general idea of how the agents are learning and working towards their maximum payouts ever obtained.

Note that it might be possible that they don't actually converge to these maximum payouts - think about this as a game: the Nash equilibria don't have to be Pareto optima. Maybe an agent's maximum payout was obtained with a comibnation of policies that the other three agents will never use again. Also, towards the end of the learning process, they tend to stick to the best policy they found during the exploration phase, so if this policy combined with the best policy of another agent leads them to start losing and losing... I'm just saying it could happen.

In [9]:
total_epochs = 100000  # 10000 epochs is a good number to play, need to find a better way to constraint
agents = [retail_agent, wholesale_agent, regional_warehouse_agent, factory_agent]

# TODO create a function that doesn't learn, only
# asks on t for what the downstream agent asked for on t-1
# this would be the equivalent of ordering upstream what the client ordered the previous day
def order_by_the_day(agent,day):
    return agent.downstream_agent.current_policy[day-1]

def create_demand(day):
    x = np.random.uniform(0, 1)
    if x < p_exploration:  # exploRation
        return random.randint(0,70)  # 70 is the maximum harvest of all year, occurs August 15th
    else:  # exploTation
        return agent.best_policy[day-1]
    
# POLICY ITERATION  ----------------------------------------------------------------  
    
start_time = time.time()    
    
for j in range(total_epochs):  
    if j % (total_epochs/20) == 0:
        print(" ")  # These last two lines are used for the printing - the just make sure every time point appears clearly separated
    p_exploration = max(0.05,(total_epochs - j) / total_epochs)  # starts in 1 ends in the first number so it always explores a bit
    day = 0
    # Reinitialize inventories and money, etc at the beginning of the year
    # The only things that should stay are best policies and payout, this is what the agent learns over time
    for agent in agents:
        agent.inventory = agent.initial_inventory
        agent.total_warehousing_costs = 0
        agent.total_money = 0
        agent.backlog = 0             
    
    while day < 365:  # one year
        day+=1
        # PART 1
        # Transactions for previous day happen. These are fixed.
        # Orders are fulfilled first time in the morning
        # Everyone gets their shippings at the same time
        # Factory
        fulfilled_to_factory = max(0,fields_agent.supply[day-1] - factory_agent.current_policy[day-1])
        factory_agent.receive_upstream(fulfilled_to_factory)
        # Regional Warehouse
        fulfilled_to_regional_warehouse = factory_agent.give_downstream(regional_warehouse_agent.current_policy[day-1])
        regional_warehouse_agent.receive_upstream(fulfilled_to_regional_warehouse)
        # Wholesale
        fulfilled_to_wholesale = regional_warehouse_agent.give_downstream(wholesale_agent.current_policy[day-1])
        wholesale_agent.receive_upstream(fulfilled_to_wholesale)
        # Retail
        fulfilled_to_retail = wholesale_agent.give_downstream(retail_agent.current_policy[day-1])
        retail_agent.receive_upstream(fulfilled_to_retail)
        # Customer
        fulfilled_to_customer = retail_agent.give_downstream(customer_agent.demand[day-1])
           
        for agent in agents:
            # PART 2
            # How much money did the agent end up with yesterday's decisions?
            agent.current_payout[day-1] = agent.total_money
            # Agent decides demand for today, which will (might) be fulfilled tomorrow
            if j == 0:
                agent_demand = 0
            else:
                agent_demand = create_demand(day)
                #agent_demand = 300  #If we want to evaluate any static policy
            agent.current_policy[day-1] = agent_demand
            # Paying for warehousing at the end of the day
            agent.pay_for_warehousing()
            if agent.current_payout[day-1] > agent.best_payout[day-1]:  # payout at the end of the year
                #print("I have found a better policy! Year %s " % (j))
                agent.best_policy[day-1] = agent.current_policy[:][day-1]  # [:] because they're mutable
                agent.best_payout[day-1] = agent.current_payout[:][day-1]
                #print("UPDATED! Year %s Agent %s current payout %s updated best payout %s" % (j, agent.name, agent.current_payout[-1], agent.best_payout[-1]))        
            
    for agent in agents:  # At the end of the year, save the final payout they got
        if j % (total_epochs/20) == 0:   # I will only print 20 views, doesn't matter how many epochs
            print("Year %s Agent %s current payout %s current best payout %s" % (j, agent.name, agent.current_payout[-1], agent.best_payout[-1]))        
        agent.historic_payout.append(agent.current_payout[-1])     

elapsed_time = time.time() - start_time
print("Total elapsed time for %s epochs : %s" % (total_epochs, elapsed_time))

 
Year 0 Agent Retail current payout 18065.6206 current best payout 18065.6206
Year 0 Agent Wholesale current payout -18200.0 current best payout -18200.0
Year 0 Agent Regional_Warehouse current payout -18200.0 current best payout -18200.0
Year 0 Agent Factory current payout -251629.409744 current best payout -251629.409744
 
Year 5000 Agent Retail current payout 1403.90996574 current best payout 18065.6206
Year 5000 Agent Wholesale current payout 1349.05917214 current best payout 2149.37850376
Year 5000 Agent Regional_Warehouse current payout 1380.88723713 current best payout 2007.8783569
Year 5000 Agent Factory current payout 1083.83487778 current best payout 1652.8384946
 
Year 10000 Agent Retail current payout -4263.00556902 current best payout 18065.6206
Year 10000 Agent Wholesale current payout 516.363329512 current best payout 2158.03681151
Year 10000 Agent Regional_Warehouse current payout 1495.81599992 current best payout 2007.8783569
Year 10000 Agent Factory current payout 11

# Thoughts:

## Best policies

Since there's a cost for warehousing, shouldn't they be learning something like "deploy all inventory first, *then* start buying"? Demand is, tops, 2 units per day - why on earth would the retail agent think it's a good idea to buy 50 units the first day? Retail best policy can be seen printing `retail_agent.best_policy`        

In [19]:
p_bp = figure(title="Best Policies", plot_height=450, plot_width=900, x_range = (0,365), y_range=(-1,70))
r_bp = p_bp.line(range(365), factory_agent.best_policy, color="seagreen")
p_bp.xaxis.axis_label = "Day"
p_bp.yaxis.axis_label = "Demand"

In [20]:
def update_policy(Agent):
    if   Agent == "Retail": agent = retail_agent
    elif Agent == "Wholesale": agent = wholesale_agent
    elif Agent == "Regional Warehouse": agent = regional_warehouse_agent
    elif Agent == "Factory": agent = factory_agent
    r_bp.data_source.data['y'] = agent.best_policy
    push_notebook()

In [21]:
show(p_bp, notebook_handle=True)

In [22]:
interact(update_policy, Agent=["Factory", "Regional Warehouse", "Wholesale", "Retail"])

## Historic payout graphs:

I'm finding many different behaviours depending a LOT on initial conditions (initial inventories, selling prices, warehousing costs, etc). With conditions

```
# Prices and Costs
# Prices of one beer at each level of the supply chain.
retail_price = 244.5
wholesale_price = 9
regional_warehouse_price = 8
factory_price = 7
field_price = 6
# Cost of holding one beer during one day on warehouse.
# Assumed to be the same for all levels
warehouse_price = 0.5
# Cost of backlog: non fulfilled orders
backlog_cost = 0.01

# Initial Inventories
retail_ininv = 100
wholesale_ininv = 100
regional_warehouse_ininv = 100
factory_ininv = 100
```

We can see the three types of convergence:

    1. To a middle point, as how a partial average (S(n)/n) would look like
    
        * For example, *retail*
        
    2. To an asyntote at the top, which would probably be interpretable as the best payout obtainable
    
        * For example, *regional warehouse*
        
    3. Some kind of loss over time - this could be interpreted as follows: since the players are only maximizing their own payouts, the game could lean towards a state in which a subset of the players start winning more and more, while the rest start losing more and more - especially if they're starting to always stick with their "best" found policy (this is, getting closer to the end of the training: less exploring and more exploiting) and can't find a way to "leave" that vicious cycle
    
        * For example, *wholesale* and *factory*

In [14]:
p_hp = figure(title="Historical Payouts", plot_height=450, plot_width=900, x_range = (0,total_epochs), y_range=(-3000,2500))
r_hp = p_hp.circle(range(total_epochs), factory_agent.historic_payout[1:], size=5, color="seagreen", alpha=0.2)
p_hp.xaxis.axis_label = "Iteration"
p_hp.yaxis.axis_label = "Obtained Payout"

In [15]:
def update_payout(Agent):
    if   Agent == "Retail": agent = retail_agent
    elif Agent == "Wholesale": agent = wholesale_agent
    elif Agent == "Regional Warehouse": agent = regional_warehouse_agent
    elif Agent == "Factory": agent = factory_agent
    r_hp.data_source.data['y'] = agent.historic_payout[1:]
    push_notebook()

In [16]:
show(p_hp, notebook_handle=True)

In [17]:
interact(update_payout, Agent=["Factory", "Regional Warehouse", "Wholesale", "Retail"])